In [2]:
import json
import regex as re 
import requests 
import sys

import os
import time


sys.path.append("C:\\Users\\7ross\\Desktop\\ishanu scripts\\repo\\recogai_\\nero\\nero-data")
import utils
import natsort
import threading
import math

import numpy as np

In [ ]:


with open(r"C:\Users\7ross\Desktop\ishanu scripts\repo\recogai_\nero\nero-data\api_data_collection\utils\topics.json","r") as in_json:
    topics = json.load(in_json)


version = ""

model = "gpt-4o"
json_name = "metadata.json"
interaction_info = {}
context_window = 190000
max_completion_length = 16384
novel_desired_length = 150000
num_novels = 300
chapter_length = 15000
#per minute
RPM_limit = 2000
ITPM_limit = 800000
OTPM_limit = 160000
requests_sent = 0
itokens = 0
otokens = 0

itoken_cost = 1.25
otoken_cost = 10.00

error_js_name = "error.json"
text_out_dir = r"generated_text"
interaction_info = {}

utils.max_threads = 5

thread_pause_semaphore = threading.Semaphore(1)
if(json_name in os.listdir(os.getcwd())):
        with open(json_name,"r") as infile:
            interaction_info = json.load(infile)
    
def request_anthropic(prompt, model, api_key, max_tokens=chapter_length):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "max_completion_tokens": max_tokens
    }
    
    print("request posted")
    try: 
        response = requests.post(url, headers=headers, json=data)
        print(f"response")
        print(f"request returned")
    except:
        print(f"error in requests post")
    response_js = response.json()
    print(response_js)
    itokens = response_js["usage"]["prompt_tokens"]
    otokens = response_js["usage"]["completion_tokens"]
    text =  response_js["choices"][0]["message"]["content"]

    if response.status_code == 200:
        print(f"request returned")
        return response_js,text,itokens,otokens
    else:
        print(f"Error: {response.status_code}, {response.text}")
        sys.exit()


def check_for_error(response, error_js_name):
    if(response["choices"][0]["finish_reason"]!= "stop"):
        print(f"invalid stop reason: {response["choices"][0]["finish_reason"]}")
        json.dump(response, error_js_name)
        sys.exit()


In [4]:

interaction_info = {}
if(not text_out_dir in os.listdir()):
    os.mkdir(text_out_dir)
    interaction_info = json.load(infile)
files = utils.get_all_files_with_extension(f"{os.getcwd()}\\{text_out_dir}",".json")
files = natsort.natsorted(files)
print(f"files: {files}")
for file in files:
    with open(f"{os.getcwd()}\\{text_out_dir}\\{file}","r") as infile:
        interaction_info = json.load(infile)
    text = interaction_info["novel_text"]
    
    index = int(re.search("(\d+)",os.path.basename(file)).group(0))
    with open(f"{os.getcwd()}\\{text_out_dir}\\gpt_4o{index}.txt","w",encoding="utf-8") as outfile:
        outfile.write(text)

<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
C:\Users\7ross\AppData\Local\Temp\ipykernel_18768\1642125011.py:13: SyntaxWarning: invalid escape sequence '\d'
  index = int(re.search("(\d+)",os.path.basename(file)).group(0))


files: ['metadata0.json', 'metadata1.json', 'metadata2.json', 'metadata3.json', 'metadata4.json', 'metadata5.json', 'metadata6.json', 'metadata7.json', 'metadata8.json', 'metadata9.json', 'metadata10.json', 'metadata11.json', 'metadata12.json', 'metadata13.json', 'metadata14.json', 'metadata15.json', 'metadata16.json', 'metadata17.json', 'metadata18.json', 'metadata19.json', 'metadata20.json', 'metadata21.json', 'metadata22.json', 'metadata23.json', 'metadata24.json', 'metadata25.json', 'metadata26.json', 'metadata27.json', 'metadata28.json', 'metadata29.json', 'metadata30.json', 'metadata31.json', 'metadata32.json', 'metadata33.json', 'metadata34.json', 'metadata35.json', 'metadata36.json', 'metadata37.json', 'metadata38.json', 'metadata39.json', 'metadata40.json', 'metadata41.json', 'metadata42.json', 'metadata43.json', 'metadata44.json', 'metadata45.json', 'metadata46.json', 'metadata47.json', 'metadata48.json', 'metadata49.json', 'metadata50.json', 'metadata51.json', 'metadata52.js

In [8]:
skip_indices = []
itokens = 0
otokens = 0
requests_sent = 0

COST_LIMIT = 250.0

itokens_semaphore = threading.Semaphore(1)
otokens_semaphore = threading.Semaphore(1)
requests_sent_semaphore = threading.Semaphore(1)
cost_dict = {}
if(os.path.exists("cost.json")):
    with open("cost.json","r") as infile:
        cost_dict = json.load(infile)
else:
    cost_dict["total_cost"] = 0
    cost_dict["total_itokens"] = 0
    cost_dict["total_otokens"] = 0
    with open("cost.json","w")as outfile:
        json.dump(cost_dict,outfile)
def update_cost_metadata(itokens, otokens):
    cost_dict["total_itokens"] += itokens
    cost_dict["total_otokens"] += otokens
    print(f"million itokens: {cost_dict["total_itokens"]/1000000.0}, estimated cost: {(cost_dict["total_itokens"]/1000000.0)*itoken_cost}")
    print(f"million otokens: {cost_dict["total_otokens"]/1000000.0}, estimated cost: {(cost_dict["total_otokens"]/1000000.0)*otoken_cost}")
    cost_dict["total_cost"] = ((cost_dict["total_itokens"]/1000000.0)*itoken_cost)+((cost_dict["total_otokens"]/1000000.0)*otoken_cost)
    with open("cost.json","w") as outfile:
        json.dump(cost_dict,outfile)
    if(cost_dict["total_cost"]>COST_LIMIT):
        return False
    else:
        return True

def check_rate_limits():
        #will block all other threads from generating until the calling thread is done waiting out the rate limiting
        global itokens
        global otokens
        global requests_sent
        with thread_pause_semaphore:
            if(itokens>ITPM_limit or otokens>OTPM_limit or requests_sent>RPM_limit):
                time.sleep(70)
                itokens = 0
                otokens = 0
                requests_sent = 0

@utils.multithreading
def generate_novel(topic:str,index:int):
    interaction_info = {}
    metadata_path = f"{os.getcwd()}\\{text_out_dir}\\metadata{index}.json"
    global itokens 
    global otokens 
    global requests_sent
    
    
    if(index in skip_indices):
        print(f"novel index is in skipped indices: {index}")
        return

    if(not os.path.exists(metadata_path)):
        interaction_info["novel_text"] = ""
        interaction_info["responses"] = []
    else:
        print(f"found metadata at path {metadata_path}")
        with open(metadata_path,"r") as infile:
            interaction_info = json.load(infile)    
    

    novel_current_length = len(interaction_info["novel_text"])
    first_prompt = f"I want you to act as a novelist writing about {topic}. The total length of the novel is about {novel_desired_length} characters and after generating each section of the novel i will tell you how much of the text you have generated so far. Try to generate text with some irregularities in order to fool ai detection software. In the novel you should include plot points characters dialogue and an overall plot. Generate only the text for this novel and do not add any annotations. Pace the plot and story according to the current number of characters I tell you. Generate the text in full paragraphs and chapters of about {chapter_length} character length. Continue generating chapter 1 until I tell you to stop. Generate EXCLUSIVELY novel text with no annotations confirmations or listing the chapter number."
    
    
    while(len(interaction_info["novel_text"])<novel_desired_length):
        prompt = ""
        if(novel_current_length == 0):
            prompt = first_prompt
        else:
            prompt = interaction_info["novel_text"] + f"Here is the previous generated text. You have generated {novel_current_length} characters so far. Generate chapter {math.floor(novel_current_length/chapter_length)+1} until i tell you to stop, also remember to generate text in complete paragraphs and do not generate lines with a single sentence. Generate EXCLUSIVELY novel text"
            
            if(len(prompt)+max_completion_length>context_window):
                prompt = prompt[len(prompt)-(context_window-max_completion_length)::]
        check_rate_limits() 
        try: 
            response,text,itk,otk = request_anthropic(prompt,model,api_key,max_tokens=max_completion_length)
        except Exception as err:
            return err
        with requests_sent_semaphore:
            requests_sent += 1 
            
        check_for_error(response,error_js_name)
        with itokens_semaphore:
            itokens += itk
        with otokens_semaphore:
            otokens += otk
        interaction_info["responses"].append(response)
        interaction_info["novel_text"] += text
        novel_current_length += len(text)
        print(f"current novel: {index}")
        print(f"response text length: {len(text)}")
        print(f"total novel length: {novel_current_length}")
        print(f"preview: \n{text[:500]}")

        with itokens_semaphore,otokens_semaphore:
            if(not update_cost_metadata(itk,otk)):
                print(f"cost limit hit")
                sys.exit()

        if(otk < 200):
            print(f"adding {index} to skipped indices")
            skip_indices.append(index)
            return None
        
        with open(metadata_path,"w") as outfile:
           json.dump(interaction_info,outfile,indent = 5) 

        
    return None



        

In [ ]:
def is_written(index:int) -> bool:
    metadata_path = f"{os.getcwd()}\\{text_out_dir}\\metadata{index}.json"
    if(os.path.exists(metadata_path)):
        interaction_info = {}
        with open(metadata_path,"r") as infile:
            interaction_info = json.load(infile)
        
        if(len(interaction_info["novel_text"])>=novel_desired_length):
            return True
        else:
            if(index in skip_indices):
                return True
            else:
                return False
    else:
        return False

def is_generation_finished():
    files = utils.get_all_files_with_extension(f"{os.getcwd()}\\{text_out_dir}",".json")
    files = natsort.natsorted(files)
    for file in files:
        index = int(re.search("(\d+)",os.path.basename(file)).group(0))
        if(not is_written(index)):
            return False


# writing = []
# for _ in range(num_novels):
#     novel_index = get_next_index()
#     if(novel_index == None):
#         break
#     else:
#         print(f"writing: {novel_index}")
#         writing.append(novel_index)
#         topic = topics[novel_index]
#         generate_novel(topic)
if(os.path.exists(f"{os.getcwd()}\\skip_indices.json")):
    with open(f"{os.getcwd()}\\skip_indices.json","r") as infile:
        skip_indices = json.load(infile)
for novel_index in range(num_novels):
    if(not is_written(novel_index)):
        topic = topics[novel_index]
        err = generate_novel(topic,novel_index)
        while(err != None):
            time.sleep(10)
            print(err)
            print("restarting generation")
            err = generate_novel(topic,novel_index)
        with open(f"{os.getcwd()}\\skip_indices.json","w") as outfile:
            json.dump(skip_indices,outfile)
    else:
        print(f"skipping index: {novel_index}")

<>:22: SyntaxWarning: invalid escape sequence '\d'
<>:22: SyntaxWarning: invalid escape sequence '\d'
C:\Users\7ross\AppData\Local\Temp\ipykernel_4644\640172611.py:22: SyntaxWarning: invalid escape sequence '\d'
  index = int(re.search("(\d+)",os.path.basename(file)).group(0))


multithread threding count: 9
found metadata at path c:\Users\7ross\Desktop\ishanu scripts\repo\recogai_\nero\nero-data\api_data_collection\openai\gpt4o\generated_text\metadata0.json
request posted
skipping index: 1
skipping index: 2
skipping index: 3
skipping index: 4
skipping index: 5
skipping index: 6
skipping index: 7
skipping index: 8
skipping index: 9
skipping index: 10
skipping index: 11
skipping index: 12
skipping index: 13
skipping index: 14
skipping index: 15
skipping index: 16
skipping index: 17
skipping index: 18
skipping index: 19
skipping index: 20
skipping index: 21
skipping index: 22
skipping index: 23
skipping index: 24
skipping index: 25
skipping index: 26
skipping index: 27
skipping index: 28
skipping index: 29
skipping index: 30
skipping index: 31
skipping index: 32
skipping index: 33
skipping index: 34
skipping index: 35
skipping index: 36
skipping index: 37
skipping index: 38
skipping index: 39
skipping index: 40
skipping index: 41
skipping index: 42
skipping inde

response
request returned
{'id': 'chatcmpl-CbgHf30qmEp8d0pwKjdu45DTlWEsE', 'object': 'chat.completion', 'created': 1763096711, 'model': 'gpt-4o-2024-08-06', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "I'm sorry, but I can't assist with that request.", 'refusal': None, 'annotations': []}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 20857, 'completion_tokens': 11, 'total_tokens': 20868, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'service_tier': 'default', 'system_fingerprint': 'fp_cbf1785567'}
request returned
current novel: 0
response text length: 48
total novel length: 111585
preview: 
I'm sorry, but I can't assist with that request.
million itokens: 102.435435, estimated cost: 128.04429375
million otokens: 7.342305, estimated cost: 73.42304999999999
adding 0 to skipped in

: 